In [58]:
import pandas as pd
import os
import json
import unidecode

# Ruta del archivo Excel
filePath = '../sdei-annual-pm2-5/sdei-annual-pm2-5-concentrations-countries-urban-areas-v1-1998-2016-xlsx.xlsx'

# Verificar si el archivo existe
if not os.path.exists(filePath):
    print(f"No se encontró el archivo: {filePath}")
    exit()

# Cargar el archivo Excel
excelData = pd.ExcelFile(filePath)

# Información de las hojas a cargar y los filtros a aplicar del excel de PM2.5
sheet_info = {
    'Country PM2.5 Exposure': {
        'filter_prefix': 'AVPMC_',
        'filter_cols': ['COUNTRY']
    },
    'Country PM2.5 Exceedance': {
        'filter_prefix': 'PMEXDC_',
        'filter_cols': ['COUNTRY']
    },
    'Urban PM2.5 Exposure': {
        'filter_prefix': 'AVPMU_',
        'filter_cols': ['COUNTRYENG']
    }
}

def load_and_filter_sheet(excel_data, sheet_name, filter_prefix, filter_cols):
    '''
    Cargar una hoja de un archivo Excel y filtrar las columnas que empiezan con un prefijo. 
    Renombrar las columnas eliminando el prefijo y espacios.
    Todo esto en dataframe de pandas.
    '''
    # Cargar la hoja
    df = excel_data.parse(sheet_name)
    # Filtrar las columnas que empiezan con el prefijo
    data_columns = [col for col in df.columns if col.startswith(filter_prefix)]
    # Renombrar las columnas eliminando el prefijo y espacios
    renamed_columns = {col: col.replace(filter_prefix, '').strip() for col in data_columns}
    df.rename(columns=renamed_columns, inplace=True)
    # Crear lista de columnas filtradas (sin prefijos)
    filtered_columns = filter_cols + list(renamed_columns.values())
    # Seleccionar las columnas filtradas
    filtered_df = df[filtered_columns]
    # Asegurar que las cadenas de texto en las columnas filtradas comiencen con mayúscula inicial y el resto en minúsculas, sin tildes ni apóstrofos
    for col in filter_cols:
        if col in filtered_df.columns:
            filtered_df.loc[:, col] = filtered_df[col].apply(lambda x: unidecode.unidecode(x).capitalize() if isinstance(x, str) else x)
    
    return filtered_df


   

# Funcion que reciba el nombre del pais y que devuelva un listado de las hojas en las que se encuentra y los años en los que se encuentra
def get_country_info_params(country):
    '''
    Función que recibe el nombre del país y devuelve un listado de las hojas en las que se encuentra y los años en los que se encuentra.
    '''
    country_info = {}
    for sheet_name, info in sheet_info.items():
        df = cargarYfiltrarhojas(excelData).get(sheet_name)
        if df is not None:
            country_col = None
            if 'COUNTRY' in df.columns:
                country_col = 'COUNTRY'
            elif 'COUNTRYENG' in df.columns:
                country_col = 'COUNTRYENG'
            else:
                continue
            if country in df[country_col].values:
                country_info[sheet_name] = df[df[country_col] == country].columns
    return country_info


# Cargar y filtrar cada hoja
def cargarYfiltrarhojas(excelData):
    '''
    Cargar y filtrar cada hoja del archivo Excel.
    Aplicar los filtros definidos en la variable sheet_info.
    el tipo de dato que se devuelve es un diccionario con el nombre de la hoja como clave y el dataframe filtrado como valor.
    '''
    filtered_dataframes = {}
    for sheet_name, info in sheet_info.items():
        filtered_df = load_and_filter_sheet(
            excelData,
            sheet_name,
            info['filter_prefix'],
            info['filter_cols']
        )
        filtered_dataframes[sheet_name] = filtered_df
    return filtered_dataframes

def get_filtered_data(excelData, sheet_name=None, country=None, year=None , nameFileJson = '../datasets/data.json'):
    """
    Devuelve los datos filtrados según los parámetros proporcionados.
    """
    results = []
    sheets_to_check = [sheet_name] if sheet_name else cargarYfiltrarhojas(excelData = excelData).keys()
    
    for sheet in sheets_to_check:
        df = cargarYfiltrarhojas(excelData = excelData).get(sheet)
        if df is not None:
            df_filtered = df.copy()
            
            # Filtrar por país
            if country is not None:
                country_col = None
                if 'COUNTRY' in df_filtered.columns:
                    country_col = 'COUNTRY'
                elif 'COUNTRYENG' in df_filtered.columns:
                    country_col = 'COUNTRYENG'
                else:
                    continue
                df_filtered = df_filtered[df_filtered[country_col] == country]
                if df_filtered.empty:
                    continue

            # Filtrar por año
            if year is not None:
                year = str(year)
                if year in df_filtered.columns:
                    columns_to_keep = [col for col in df_filtered.columns if not col.isdigit()] + [year]
                    df_filtered = df_filtered[columns_to_keep]
                else:
                    continue
            else:
                # Incluir todas las columnas de años
                year_columns = [col for col in df_filtered.columns if col.isdigit()]
                columns_to_keep = [col for col in df_filtered.columns if not col.isdigit()] + year_columns
                df_filtered = df_filtered[columns_to_keep]
            
            # Añadir el nombre de la hoja
            df_filtered['Sheet'] = sheet
            results.append(df_filtered)
    
    if results:
        df_result = pd.concat(results, ignore_index=True)
        return df_result.to_json(orient='records', path_or_buf= nameFileJson)
    else:
        return json.dumps([])


In [59]:
# Ejemplos de uso:
get_country_info_params('Argentina')

/home/juan27/.local/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/home/juan27/.local/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/home/juan27/.local/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


{'Country PM2.5 Exposure': Index(['COUNTRY', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
        '2015'],
       dtype='object'),
 'Country PM2.5 Exceedance': Index(['COUNTRY', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
        '2015'],
       dtype='object'),
 'Urban PM2.5 Exposure': Index(['COUNTRYENG', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
        '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
        '2014', '2015', '2016'],
       dtype='object')}

In [66]:
# 2. Filtrar por país
json_por_pais = get_filtered_data(country='Colombia')
print("\nDatos para Colombia:")
print(json_por_pais)


Datos para Colombia:
[{"COUNTRY":"Colombia","2008":6.5176484937,"2009":7.4727359403,"2010":9.7057362387,"2011":6.3935288919,"2012":9.9906223802,"2013":8.2327534953,"2014":7.6382660527,"2015":7.1180866308,"Sheet":"Country PM2.5 Exposure","COUNTRYENG":null,"1998":null,"1999":null,"2000":null,"2001":null,"2002":null,"2003":null,"2004":null,"2005":null,"2006":null,"2007":null,"2016":null},{"COUNTRY":"Colombia","2008":2.4556198785,"2009":2.8382164135,"2010":8.6384392154,"2011":2.3437387644,"2012":8.8354609218,"2013":5.9500808999,"2014":2.9659934105,"2015":2.6041372727,"Sheet":"Country PM2.5 Exceedance","COUNTRYENG":null,"1998":null,"1999":null,"2000":null,"2001":null,"2002":null,"2003":null,"2004":null,"2005":null,"2006":null,"2007":null,"2016":null},{"COUNTRY":null,"2008":0.9434782681,"2009":1.1521739182,"2010":0.7521739136,"2011":0.8217391268,"2012":1.0913043515,"2013":0.9043478266,"2014":0.9434782681,"2015":0.939130441,"Sheet":"Urban PM2.5 Exposure","COUNTRYENG":"Colombia","1998":1.6391

In [67]:
# 3. Filtrar por hoja
json_por_hoja = get_filtered_data(sheet_name='Country PM2.5 Exposure')
print("\nDatos de la hoja 'Country PM2.5 Exposure':")
print(json_por_hoja)


Datos de la hoja 'Country PM2.5 Exposure':
[{"COUNTRY":"Afghanistan","2008":13.2613703711,"2009":10.5070193678,"2010":11.9254917993,"2011":11.9669335239,"2012":13.4089859372,"2013":13.0235426258,"2014":12.2029002695,"2015":15.2331373925,"Sheet":"Country PM2.5 Exposure"},{"COUNTRY":"Albania","2008":9.9145624185,"2009":9.0140325231,"2010":9.7203388625,"2011":9.9902603705,"2012":10.2651756651,"2013":9.6878014197,"2014":9.3546532804,"2015":11.8006311055,"Sheet":"Country PM2.5 Exposure"},{"COUNTRY":"Algeria","2008":6.1386496386,"2009":4.6626322559,"2010":5.4414866863,"2011":5.2902802425,"2012":5.601431021,"2013":5.3973196964,"2014":6.053068689,"2015":8.6235388136,"Sheet":"Country PM2.5 Exposure"},{"COUNTRY":"American Samoa","2008":0.0996477096,"2009":0.0995110127,"2010":0.0995110127,"2011":0.0995110127,"2012":0.0995110127,"2013":0.1012694136,"2014":0.1990220253,"2015":0.2972666079,"Sheet":"Country PM2.5 Exposure"},{"COUNTRY":"Andorra","2008":5.6950730744,"2009":5.4800212025,"2010":5.173324

In [68]:

# 4. Filtrar por país y año
json_por_pais_ano = get_filtered_data(country='Colombia', year=2015)
print("\nDatos para Colombia en 2015:")
print(json_por_pais_ano)


Datos para Colombia en 2015:
[{"COUNTRY":"Colombia","2015":7.1180866308,"Sheet":"Country PM2.5 Exposure","COUNTRYENG":null},{"COUNTRY":"Colombia","2015":2.6041372727,"Sheet":"Country PM2.5 Exceedance","COUNTRYENG":null},{"COUNTRY":null,"2015":0.939130441,"Sheet":"Urban PM2.5 Exposure","COUNTRYENG":"Colombia"},{"COUNTRY":null,"2015":1.4339999843,"Sheet":"Urban PM2.5 Exposure","COUNTRYENG":"Colombia"},{"COUNTRY":null,"2015":2.1616822214,"Sheet":"Urban PM2.5 Exposure","COUNTRYENG":"Colombia"},{"COUNTRY":null,"2015":2.3420289755,"Sheet":"Urban PM2.5 Exposure","COUNTRYENG":"Colombia"},{"COUNTRY":null,"2015":3.2020334817,"Sheet":"Urban PM2.5 Exposure","COUNTRYENG":"Colombia"},{"COUNTRY":null,"2015":3.0508009116,"Sheet":"Urban PM2.5 Exposure","COUNTRYENG":"Colombia"},{"COUNTRY":null,"2015":2.7960244627,"Sheet":"Urban PM2.5 Exposure","COUNTRYENG":"Colombia"},{"COUNTRY":null,"2015":3.0,"Sheet":"Urban PM2.5 Exposure","COUNTRYENG":"Colombia"},{"COUNTRY":null,"2015":2.921000011,"Sheet":"Urban PM2

In [6]:
# 5. Filtrar por país, año y hoja
json_por_pais_ano_hoja = get_filtered_data(excelData,sheet_name='Country PM2.5 Exposure', country='Colombia', year=2015)
print("\nDatos para Colombia en 2015 en la hoja 'Country PM2.5 Exposure':")
print(json_por_pais_ano_hoja)

/home/juan27/.local/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")



Datos para Colombia en 2015 en la hoja 'Country PM2.5 Exposure':
None


In [37]:
# 6. Filtrar por país, año y hoja distinta
json_por_pais_ano_hoja = get_filtered_data(excelData,sheet_name='Country PM2.5 Exposure', year=2015)
print("\nDatos para Colombia en 2015 en la hoja 'Country PM2.5 Exposure':")
print(json_por_pais_ano_hoja)

/home/juan27/.local/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")



Datos para Colombia en 2015 en la hoja 'Country PM2.5 Exposure':
None


In [32]:
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import shape, MultiPolygon
from shapely.ops import transform
import topojson as tp

# Cargar el archivo Excel
filePath = '../sdei-annual-pm2-5/sdei-annual-pm2-5-concentrations-countries-urban-areas-v1-1998-2016-xlsx.xlsx'
excelData = pd.ExcelFile(filePath)

# Cargar archivos JSON de ubicaciones de todos los continentes
data_files = [
    '../../topojson-maps/continents/south-america.json',
    '../../topojson-maps/continents/oceania.json',
    '../../topojson-maps/continents/north-america.json',
    '../../topojson-maps/continents/europe.json',
    '../../topojson-maps/continents/asia.json',
    '../../topojson-maps/continents/africa.json'
]

# Leer todos los archivos JSON al mismo tiempo y combinarlos en una lista
geojson_data = []
for file_path in data_files:
    with open(file_path, 'r') as f:
        topo_data = json.load(f)
        # Obtener el primer objeto del TopoJSON para convertir a GeoJSON
        object_name = list(topo_data.get('objects', {}).keys())[0] if 'objects' in topo_data else None
        if object_name:
            geojson_data.extend(tp.Topology(topo_data).to_geojson()['features'])

# Extraer las geometrías y nombres de los países
country_locations = {}
for country in geojson_data:
    country_name = country['properties']['geounit']  # Nombre del país
    country_geometry = shape(country['geometry'])   # Geometría del país
    country_locations[country_name] = country_geometry  # Almacena la geometría usando el nombre del país como clave

# Función para cargar y filtrar las hojas del archivo Excel
def load_and_filter_sheet(excel_data, sheet_name, filter_prefix, filter_cols):
    df = excel_data.parse(sheet_name)
    data_columns = [col for col in df.columns if col.startswith(filter_prefix)]
    renamed_columns = {col: col.replace(filter_prefix, '').strip() for col in data_columns}
    df.rename(columns=renamed_columns, inplace=True)
    filtered_columns = filter_cols + list(renamed_columns.values())
    filtered_df = df[filtered_columns]
    return filtered_df

# Información de las hojas y los filtros del Excel
sheet_info = {
    'Country PM2.5 Exposure': {
        'filter_prefix': 'AVPMC_',
        'filter_cols': ['COUNTRY']
    },
    'Country PM2.5 Exceedance': {
        'filter_prefix': 'PMEXDC_',
        'filter_cols': ['COUNTRY']
    },
    'Urban PM2.5 Exposure': {
        'filter_prefix': 'AVPMU_',
        'filter_cols': ['COUNTRYENG']
    }
}

# Cargar y filtrar cada hoja
def cargarYfiltrarhojas(excelData):
    filtered_dataframes = {}
    for sheet_name, info in sheet_info.items():
        filtered_df = load_and_filter_sheet(
            excelData,
            sheet_name,
            info['filter_prefix'],
            info['filter_cols']
        )
        filtered_dataframes[sheet_name] = filtered_df
    return filtered_dataframes

# Añadir datos de ubicación a los datos de PM2.5
def get_filtered_data_with_location(excelData, sheet_name=None, country=None, year=None, nameFileJson='../datasets/data_with_location.json'):
    results = []
    filtered_dataframes = cargarYfiltrarhojas(excelData=excelData)
    sheets_to_check = [sheet_name] if sheet_name else filtered_dataframes.keys()

    for sheet in sheets_to_check:
        df = filtered_dataframes.get(sheet)
        if df is not None:
            df_filtered = df.copy()

            # **Determinar la columna de país independientemente de si 'country' es None o no**
            country_col = None
            if 'COUNTRY' in df_filtered.columns:
                country_col = 'COUNTRY'
            elif 'COUNTRYENG' in df_filtered.columns:
                country_col = 'COUNTRYENG'
            else:
                continue  # Si no hay columna de país, saltar esta hoja

            # Filtrar por país si se proporciona
            if country is not None:
                df_filtered = df_filtered[df_filtered[country_col] == country]
                if df_filtered.empty:
                    continue

            # Filtrar por año
            if year is not None:
                year = str(year)
                if year in df_filtered.columns:
                    columns_to_keep = [col for col in df_filtered.columns if not col.isdigit()] + [year]
                    df_filtered = df_filtered[columns_to_keep]
                else:
                    continue
            else:
                year_columns = [col for col in df_filtered.columns if col.isdigit()]
                columns_to_keep = [col for col in df_filtered.columns if not col.isdigit()] + year_columns
                df_filtered = df_filtered[columns_to_keep]
            
            # Añadir el nombre de la hoja al DataFrame
            df_filtered['Sheet'] = sheet

            # Añadir propiedades de ubicación (geometría)
            for index, row in df_filtered.iterrows():
                country_key = row.get(country_col)
                location = country_locations.get(country_key)
                if location:
                    df_filtered.at[index, 'geometry'] = location
                else:
                    print(f"Geometría no encontrada para {country_key}")
                    continue

            results.append(df_filtered)

    if results:
        df_result = pd.concat(results, ignore_index=True)
        gdf_result = gpd.GeoDataFrame(df_result, geometry='geometry')
        # Guardar los datos como un archivo GeoJSON
        gdf_result.to_file(nameFileJson, driver='GeoJSON')
        return gdf_result
    else:
        return gpd.GeoDataFrame(columns=['geometry'])

SystemExit: 'data' is not an object name in your topojson file

In [25]:
# Llamada a la función con un ejemplo para obtener datos con ubicación
data_with_location = get_filtered_data_with_location(excelData, sheet_name='Country PM2.5 Exposure')
print(data_with_location.head())

/home/juan27/.local/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Geometría no encontrada para Afghanistan
Geometría no encontrada para Albania
Geometría no encontrada para Algeria
Geometría no encontrada para American Samoa
Geometría no encontrada para Andorra
Geometría no encontrada para Angola
Geometría no encontrada para Anguilla
Geometría no encontrada para Antigua and Barbuda
Geometría no encontrada para Armenia
Geometría no encontrada para Aruba
Geometría no encontrada para Australia
Geometría no encontrada para Austria
Geometría no encontrada para Azerbaijan
Geometría no encontrada para Bahamas
Geometría no encontrada para Bahrain
Geometría no encontrada para Bangladesh
Geometría no encontrada para Barbados
Geometría no encontrada para Belarus
Geometría no encontrada para Belgium
Geometría no encontrada para Belize
Geometría no encontrada para Benin
Geometría no encontrada para Bermuda
Geometría no encontrada para Bhutan
Geometría no encontrada para Bosnia and Herzegovina
Geometría no encontrada para Botswana
Geometría no encontrada para Brit

/home/juan27/.local/lib/python3.10/site-packages/pyogrio/geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(


In [61]:
import pandas as pd
import os
import json
import unidecode
import plotly.express as px  # Importar Plotly Express para visualizaciones
import plotly.graph_objects as go

# Ruta del archivo Excel
filePath = '../sdei-annual-pm2-5/sdei-annual-pm2-5-concentrations-countries-urban-areas-v1-1998-2016-xlsx.xlsx'

# Verificar si el archivo existe
if not os.path.exists(filePath):
    print(f"No se encontró el archivo: {filePath}")
    exit()

# Cargar el archivo Excel
excelData = pd.ExcelFile(filePath)

# Información de las hojas a cargar y los filtros a aplicar del excel de PM2.5
sheet_info = {
    'Country PM2.5 Exposure': {
        'filter_prefix': 'AVPMC_',
        'filter_cols': ['COUNTRY']
    },
    'Country PM2.5 Exceedance': {
        'filter_prefix': 'PMEXDC_',
        'filter_cols': ['COUNTRY']
    },
    'Urban PM2.5 Exposure': {
        'filter_prefix': 'AVPMU_',
        'filter_cols': ['COUNTRYENG']
    }
}

def load_and_filter_sheet(excel_data, sheet_name, filter_prefix, filter_cols):
    '''
    Cargar una hoja de un archivo Excel y filtrar las columnas que empiezan con un prefijo. 
    Renombrar las columnas eliminando el prefijo y espacios.
    Todo esto en un DataFrame de pandas.
    '''
    # Cargar la hoja
    df = excel_data.parse(sheet_name)
    # Filtrar las columnas que empiezan con el prefijo
    data_columns = [col for col in df.columns if col.startswith(filter_prefix)]
    # Renombrar las columnas eliminando el prefijo y espacios
    renamed_columns = {col: col.replace(filter_prefix, '').strip() for col in data_columns}
    df.rename(columns=renamed_columns, inplace=True)
    # Crear lista de columnas filtradas (sin prefijos)
    filtered_columns = filter_cols + list(renamed_columns.values())
    # Seleccionar las columnas filtradas
    filtered_df = df[filtered_columns]
    # Asegurar que las cadenas de texto en las columnas filtradas comiencen con mayúscula inicial y el resto en minúsculas, sin tildes ni apóstrofos
    for col in filter_cols:
        if col in filtered_df.columns:
            filtered_df.loc[:, col] = filtered_df[col].apply(lambda x: unidecode.unidecode(x).capitalize() if isinstance(x, str) else x)
    return filtered_df

# Función que recibe el nombre del país y devuelve un listado de las hojas en las que se encuentra y los años en los que se encuentra
def get_country_info_params(country):
    '''
    Función que recibe el nombre del país y devuelve un listado de las hojas en las que se encuentra y los años en los que se encuentra.
    '''
    country_info = {}
    for sheet_name, info in sheet_info.items():
        df = cargarYfiltrarhojas(excelData).get(sheet_name)
        if df is not None:
            country_col = None
            if 'COUNTRY' in df.columns:
                country_col = 'COUNTRY'
            elif 'COUNTRYENG' in df.columns:
                country_col = 'COUNTRYENG'
            else:
                continue
            if country in df[country_col].values:
                years = [col for col in df.columns if col.isdigit()]
                country_info[sheet_name] = years
    return country_info

# Cargar y filtrar cada hoja
def cargarYfiltrarhojas(excelData):
    '''
    Cargar y filtrar cada hoja del archivo Excel.
    Aplicar los filtros definidos en la variable sheet_info.
    El tipo de dato que se devuelve es un diccionario con el nombre de la hoja como clave y el DataFrame filtrado como valor.
    '''
    filtered_dataframes = {}
    for sheet_name, info in sheet_info.items():
        filtered_df = load_and_filter_sheet(
            excelData,
            sheet_name,
            info['filter_prefix'],
            info['filter_cols']
        )
        filtered_dataframes[sheet_name] = filtered_df
    return filtered_dataframes

def get_filtered_data(excelData, sheet_name=None, country=None, year=None , nameFileJson=None):
    """
    Devuelve los datos filtrados según los parámetros proporcionados.
    """
    results = []
    filtered_dataframes = cargarYfiltrarhojas(excelData)
    sheets_to_check = [sheet_name] if sheet_name else filtered_dataframes.keys()
    
    for sheet in sheets_to_check:
        df = filtered_dataframes.get(sheet)
        if df is not None:
            df_filtered = df.copy()
            
            # Filtrar por país
            country_col = None
            if 'COUNTRY' in df_filtered.columns:
                country_col = 'COUNTRY'
            elif 'COUNTRYENG' in df_filtered.columns:
                country_col = 'COUNTRYENG'
            else:
                continue

            if country is not None:
                df_filtered = df_filtered[df_filtered[country_col] == country]
                if df_filtered.empty:
                    continue

            # Filtrar por año
            if year is not None:
                year = str(year)
                if year in df_filtered.columns:
                    columns_to_keep = [col for col in df_filtered.columns if not col.isdigit()] + [year]
                    df_filtered = df_filtered[columns_to_keep]
                else:
                    continue
            else:
                # Incluir todas las columnas de años
                year_columns = [col for col in df_filtered.columns if col.isdigit()]
                columns_to_keep = [col for col in df_filtered.columns if not col.isdigit()] + year_columns
                df_filtered = df_filtered[columns_to_keep]
            
            # Añadir el nombre de la hoja
            df_filtered['Sheet'] = sheet
            results.append(df_filtered)
    
    if results:
        df_result = pd.concat(results, ignore_index=True)
        if nameFileJson:
            df_result.to_json(orient='records', path_or_buf=nameFileJson)
        return df_result
    else:
        return pd.DataFrame()

# ----------------------------------------
# Funciones para graficar los datos
# ----------------------------------------

def plot_pm25_over_time(df, country, sheet_name='Country PM2.5 Exposure'):
    """
    Genera una gráfica interactiva de la exposición a PM2.5 a lo largo del tiempo para un país específico.
    Parámetros:
    - df: DataFrame que contiene los datos filtrados.
    - country: Nombre del país.
    - sheet_name: Nombre de la hoja de donde se obtienen los datos (por defecto 'Country PM2.5 Exposure').
    """
    # Obtener los datos filtrados para el país y la hoja especificada
    df_country = df[(df['Sheet'] == sheet_name) & (df['COUNTRY'] == country)]
    if df_country.empty:
        print(f"No se encontraron datos para {country} en {sheet_name}.")
        return

    # Transformar el DataFrame a formato largo (long format)
    year_columns = [col for col in df_country.columns if col.isdigit()]
    df_melted = df_country.melt(id_vars=['COUNTRY', 'Sheet'], value_vars=year_columns, var_name='Year', value_name='PM2.5')

    # Convertir el año a entero y PM2.5 a numérico
    df_melted['Year'] = df_melted['Year'].astype(int)
    df_melted['PM2.5'] = pd.to_numeric(df_melted['PM2.5'], errors='coerce')

    # Crear la gráfica interactiva
    fig = px.line(df_melted, x='Year', y='PM2.5', title=f'Exposición a PM2.5 en {country} a lo largo del tiempo', labels={'PM2.5': 'PM2.5 (µg/m³)'})
    fig.update_layout(xaxis_title='Año', yaxis_title='Concentración de PM2.5 (µg/m³)')
    fig.show()

def plot_pm25_comparison(df, year, sheet_name='Country PM2.5 Exposure', top_n=10):
    """
    Genera una gráfica de barras comparando la exposición a PM2.5 entre países en un año específico.
    Parámetros:
    - df: DataFrame que contiene los datos filtrados.
    - year: Año a comparar.
    - sheet_name: Nombre de la hoja de donde se obtienen los datos (por defecto 'Country PM2.5 Exposure').
    - top_n: Número de países a mostrar (por defecto 10).
    """
    # Obtener los datos filtrados para la hoja especificada y el año
    df_year = df[df['Sheet'] == sheet_name]
    year = str(year)
    if year not in df_year.columns:
        print(f"No se encontraron datos para el año {year} en {sheet_name}.")
        return

    df_year = df_year[['COUNTRY', year]].copy()
    df_year.rename(columns={year: 'PM2.5'}, inplace=True)
    df_year['PM2.5'] = pd.to_numeric(df_year['PM2.5'], errors='coerce')
    df_year.dropna(subset=['PM2.5'], inplace=True)

    # Obtener los top_n países con mayor exposición
    df_top = df_year.sort_values('PM2.5', ascending=False).head(top_n)

    # Crear la gráfica interactiva
    fig = px.bar(df_top, x='COUNTRY', y='PM2.5', title=f'Top {top_n} países con mayor exposición a PM2.5 en {year}', labels={'PM2.5': 'PM2.5 (µg/m³)'})
    fig.update_layout(xaxis_title='País', yaxis_title='Concentración de PM2.5 (µg/m³)')
    fig.show()

def plot_pm25_world_map(df, year, sheet_name='Country PM2.5 Exposure'):
    """
    Genera un mapa interactivo mostrando la exposición a PM2.5 por país en un año específico.
    Parámetros:
    - df: DataFrame que contiene los datos filtrados.
    - year: Año a visualizar.
    - sheet_name: Nombre de la hoja de donde se obtienen los datos (por defecto 'Country PM2.5 Exposure').
    """
    # Obtener los datos filtrados para la hoja especificada y el año
    df_year = df[df['Sheet'] == sheet_name]
    year = str(year)
    if year not in df_year.columns:
        print(f"No se encontraron datos para el año {year} en {sheet_name}.")
        return

    df_year = df_year[['COUNTRY', year]].copy()
    df_year.rename(columns={year: 'PM2.5'}, inplace=True)
    df_year['PM2.5'] = pd.to_numeric(df_year['PM2.5'], errors='coerce')
    df_year.dropna(subset=['PM2.5'], inplace=True)

    # Crear el mapa interactivo
    fig = px.choropleth(df_year, locations='COUNTRY', locationmode='country names', color='PM2.5',
                        color_continuous_scale='YlOrRd', title=f'Exposición a PM2.5 en el mundo en {year}',
                        labels={'PM2.5': 'PM2.5 (µg/m³)'})
    fig.update_layout(geo=dict(showframe=False, showcoastlines=True))
    fig.show()

def plot_pm25_multiple_countries(df, countries, sheet_name='Country PM2.5 Exposure'):
    """
    Genera una gráfica interactiva comparando la exposición a PM2.5 a lo largo del tiempo para múltiples países.
    Parámetros:
    - df: DataFrame que contiene los datos filtrados.
    - countries: Lista de nombres de países.
    - sheet_name: Nombre de la hoja de donde se obtienen los datos (por defecto 'Country PM2.5 Exposure').
    """
    # Obtener los datos filtrados para los países y la hoja especificada
    df_countries = df[(df['Sheet'] == sheet_name) & (df['COUNTRY'].isin(countries))]
    if df_countries.empty:
        print(f"No se encontraron datos para los países especificados en {sheet_name}.")
        return

    # Transformar el DataFrame a formato largo (long format)
    year_columns = [col for col in df_countries.columns if col.isdigit()]
    df_melted = df_countries.melt(id_vars=['COUNTRY', 'Sheet'], value_vars=year_columns, var_name='Year', value_name='PM2.5')

    # Convertir el año a entero y PM2.5 a numérico
    df_melted['Year'] = df_melted['Year'].astype(int)
    df_melted['PM2.5'] = pd.to_numeric(df_melted['PM2.5'], errors='coerce')

    # Crear la gráfica interactiva
    fig = px.line(df_melted, x='Year', y='PM2.5', color='COUNTRY', title='Exposición a PM2.5 a lo largo del tiempo por país', labels={'PM2.5': 'PM2.5 (µg/m³)'})
    fig.update_layout(xaxis_title='Año', yaxis_title='Concentración de PM2.5 (µg/m³)')
    fig.show()

# ----------------------------------------
# Ejemplos de uso
# ----------------------------------------

if __name__ == "__main__":
    # Obtener los datos filtrados (puedes ajustar los parámetros según tus necesidades)
    df_data = get_filtered_data(excelData)

    # Ejemplo 1: Graficar la exposición a PM2.5 a lo largo del tiempo para Colombia
    plot_pm25_over_time(df_data, country='Colombia')

    # Ejemplo 2: Comparar la exposición a PM2.5 entre países en 2015
    plot_pm25_comparison(df_data, year=2015, top_n=10)

    # Ejemplo 3: Mostrar un mapa mundial de la exposición a PM2.5 en 2015
    plot_pm25_world_map(df_data, year=2015)

    # Ejemplo 4: Comparar la exposición a PM2.5 a lo largo del tiempo para varios países
    countries_list = ['Colombia', 'Argentina', 'Brazil', 'Chile', 'Peru']
    plot_pm25_multiple_countries(df_data, countries=countries_list)


/home/juan27/.local/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
